still need to get sentences from books 1,2,6

In [1]:
from bs4 import BeautifulSoup
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize
import bs4

In [2]:
def make_list(soup):
    l = []
    for n in soup:
        try:
            l.append(int(n['n']))
        except:
            l.append(-1)  
    return remove_duplicates(l)


def remove_duplicates(l):
    i = 0
    while i+1 < len(l):
        if l[i] == l[i+1]:
            l[i+1] += 1
        if l[i] > l[i+1] and l[i+1] != -1:
            l[i] = l[i+1]-1
        i += 1
    return l

def match(large,small):
    l = make_list(large)
    s = make_list(small)
    li = 0
    si = 0
    groups = []
    #print(l,s)
    while si < len(s) and li < len(l):
        if s[si] == -1:
            si += 1
            continue
        if l[li] == -1:
            li += 1
            continue
        lgroup = []
        sgroup = []
        if l[li] == s[si]:
            lgroup.append(li)
            sgroup.append(si)
              
        if si+1 < len(s) and li+1 < len(l): 
            if s[si+1] == -1:
                si += 1
            
            if l[li+1] == -1:
                li += 1
            
            
            if l[li+1] != s[si+1]:
                while l[li+1] != s[si+1]:
                    if si+1 < len(s) and li+1 < len(l):
                        if l[li+1] > s[si+1]:
                            sgroup.append(si+1)
                            si += 1
                            if s[si] == -1:
                                if si+1 < len(s):
                                    si += 1
                                else:
                                    pass
                        else:
                            lgroup.append(li+1)
                            li += 1
                            if l[li] == -1:
                                if li+1 < len(l):
                                    li += 1
                                else:
                                    pass
                    else:
                        break
                si += 1
                li += 1
                groups.append([sgroup,lgroup])
            else:
                si += 1
                li += 1
                groups.append([sgroup,lgroup]) 
        else:
            break        
    
    groups.append([sgroup,lgroup])
    return groups
        

def align_soups(m,small,large):
    s_alignments = []
    l_alignments = []
    for g in m:
        s = []
        l = []
        for i in g[0]:
            s.append(small[i])
        for i in g[1]:
            l.append(large[i])
        s_alignments.append(s)
        l_alignments.append(l)   
    return s_alignments,l_alignments


def get_pairs(latin_soup, eng_soup, tag, typename, typos):
    """ Given two soups, returns soups of equal size, with each element corresponding to the same
        element in the other soup.
        Lists may contain further lists of soups which are aligned with one another, they are combined into one list
        in the first step of this function.
    """
    lat_sect,eng_sect = [], []
    
    for i in range(len(latin_soup)):
        lat_sect += latin_soup[i].findAll(tag, {typename: typos})
    for i in range(len(eng_soup)):
        eng_sect += eng_soup[i].findAll(tag, {typename: typos})

    if len(lat_sect) > len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].parent['n'])
            m = match(lat_sect,eng_sect)
            eng_alignments,lat_alignments = align_soups(m,eng_sect,lat_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    elif len(lat_sect) < len(eng_sect):
        try:
            #print(latin_soup[0]['n'],latin_soup[0].name)
            m = match(eng_sect,lat_sect)
            lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
            return lat_alignments,eng_alignments
        except:
            print("error with {}, {}".format(tag,typos))
    else:
        if eng_sect[0]['n'] == 'intro':
            try:
                #print(latin_soup[0]['n'],latin_soup[0].name)
                m = match(eng_sect,lat_sect)
                lat_alignments,eng_alignments = align_soups(m,lat_sect,eng_sect)
                return lat_alignments,eng_alignments
            except:
                print("error with {}, {}".format(tag,typos))
        else:
            
            return lat_sect, eng_sect
    



def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          

def get_text(soup):
    """Expects a soup in the form of a find('p') selection"""
    s = []
    for c in soup.children:
        if c.name in [None,'corr', 'name']:
            try:
                s.append(c.text.strip())
            except:
                s.append(c.strip())
    return s


def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

def get_sentences(chap_soup, sect_soup):
    t = bs4.element.Tag
    sect = []
    sentences = []
    i = 0
    c = 0
    children = list(chap_soup.children)
    soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
    c += 1
    while type(soup) != t:
        soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
        c += 1
    i += 1
    while c < len(children):
        print(sentences)
        while soup.next_sibling != None:
            soup = soup.next_sibling
            if soup.name == 'milestone':
                try:
                    if int(soup['n']) == int(sect_soup[i]['n']):
                        i += 1
                        sentences.append(' '.join(sect))
                        sect = []
                except:
                    soup = soup.next_sibling
                    i += 1
            if soup.name in ['corr', 'name','hi','quote','reg', 'l', 'lg', 'placeName']:
                    sect.append(soup.text.strip())
            elif soup.name == None:
                    sect.append(soup.strip())
                    
        sentences.append(' '.join(sect))
        sect = []
        try:
            soup = children[c].find('milestone')#, {'n':sect_soup[i]['n']})
            c += 1
            while type(soup) != t and c < len(children):
                soup = children[c].find('milestone')
                c += 1
            i += 1
        except:
            continue
    sentences.append(' '.join(sect))
    sect = []
    return sentences

In [5]:
t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

f = open('../Latin/Caesar/opensource/caes.bc_lat.xml')
latin_soup = BeautifulSoup(f.read(),'lxml')
f.close()

f = open('../Latin/Caesar/opensource/caes.bc_eng.xml')
eng_soup = BeautifulSoup(f.read(),'lxml')
f.close()

In [6]:
lat,eng = get_pairs([latin_soup],[eng_soup],'div1','type','book')

lchaps = []
echaps = []
for i in range(len(lat)):
    try:
        p = get_pairs([lat[i]],[eng[i]],'div2','type','chapter')
        if type(p[0][0]) == list:
            lchaps.append([s[0] for s in p[0]])
            echaps.append([s[0] for s in p[1]])
        else:
            lchaps.append(p[0])
            echaps.append(p[1])
    except:
        print('error with {}'.format(i))

alignments = []
for b in range(len(lat)):
    print('b: ',b)
    for c in range(len(lchaps[b])):
        es = ' '.join(echaps[b][c].text.replace('\n',' ').split())
        ls = lchaps[b][c].text.replace('\n', ' ')

        ls = re.sub(r'\. \. \.','',ls)
        es = re.sub(r'\. \. \.','',es)
        ls = re.sub('[:;,]','',ls)
        es = re.sub('[:;,]','',es)

        epar = sent_tokenize(es)
        lpar = s_tokenizer.tokenize_sentences(ls)
        alignments.append(get_sec_alignments(lpar,epar))
        #sec_alignments.append(get_sec_alignments(lpar[p],epar[p]))
        #alignments.append(sec_alignments)

error with 0
b:  0
b:  1
b:  2


IndexError: list index out of range

In [7]:
p = get_pairs([lat[0]],[eng[0]],'div2','type','chapter')

In [8]:
# ignoring first chapter of first book, some sort of reading error occured. 

lch_1 = p[0][1:]
ech_1 = p[1][1:]

In [9]:
len(lch_1[0])

1

In [10]:
es = ' '.join(ech_1[0][0].text.replace('\n',' ').split())
ls = lch_1[0][0].text.replace('\n', ' ')

ls = re.sub(r'\. \. \.','',ls)
es = re.sub(r'\. \. \.','',es)
ls = re.sub('[:;,]','',ls)
es = re.sub('[:;,]','',es)

epar = sent_tokenize(es)
lpar = s_tokenizer.tokenize_sentences(ls)

In [11]:
for c in range(len(lch_1)):
    es = ' '.join(ech_1[c][0].text.replace('\n',' ').split())
    ls = lch_1[c][0].text.replace('\n', ' ')

    ls = re.sub(r'\. \. \.','',ls)
    es = re.sub(r'\. \. \.','',es)
    ls = re.sub('[:;,]','',ls)
    es = re.sub('[:;,]','',es)

    epar = sent_tokenize(es)
    lpar = s_tokenizer.tokenize_sentences(ls)
    alignments.append(get_sec_alignments(lpar,epar))

In [12]:
count = 0
for a in alignments:
    for s in a:
        count += len(s)

In [26]:
alignments[-2][0]

('  Id vero militibus fuit pergratum et iucundum ut  ex ipsa significatione cognosci potuit ut qui aliquid iusti incommodi expectavissent ultro praemium missionis ferrent.',
 ["These conditions were agreeable to Afranius's soldiers who instead of being punished as they feared were in some sort rewarded by the discharge procured them."])

In [27]:
count

2566

In [14]:
alignments[5]

[(['  Commisso proelio Massiliensibus res nulla ad  virtutem defuit.'],
  'Accordingly in the engagement they behaved with the most determined courage.'),
 (['sed memores eorum praeceptorum quae paulo ante ab suis acceperant hoc animo decertabant ut nullum aliud tempus ad conandum habituri viderentur et quibus in pugna vitae periculum accideret non ita multo se reliquorum civium fatum antecedere existimarent quibus urbe capta eadem esset   belli fortuna patienda.'],
  'The remembrance of what their wives and children had represented to them at their departure served to exalt their bravery in a full persuasion that this was the last opportunity they should have of exerting themselves in defence of their country and that if they fell in the engagement their fellow-citizens could not long survive them as their fate must be the same upon the taking of the town.'),
 (['diductisque nostris paulatim navibus et artificio gubernatorum et mobilitati navium locus dabatur et siquando nostri facult

In [28]:
f = open('aligned_sentences/caes_bc_sentences.json','w') 
j = json.dumps(alignments)
f.write(j)
f.close()

In [30]:
f = open('../Latin/Caesar/opensource/caes.bg_lat.xml')
latin_soup = BeautifulSoup(f.read(),'lxml')
f.close()

f = open('../Latin/Caesar/opensource/caes.bg_eng.xml')
eng_soup = BeautifulSoup(f.read(),'lxml')
f.close()

In [31]:
len(eng_soup.findAll('div1',{'type':'Book'}))

8

In [32]:
lat,eng = get_pairs([latin_soup],[eng_soup],'div1','type','Book')

In [ ]:
0,3,4,5,7

In [33]:
len(eng[2].findAll('p'))

29

In [34]:
len(lat[2].findAll('p'))

27

In [35]:
lpars = []
for p in lat[0].findAll('p'):
    ls = p.text.replace('\n', ' ')

    ls = re.sub(r'\. \. \.','',ls)
    ls = re.sub('[:;,]','',ls)
    lpars.append(len(s_tokenizer.tokenize_sentences(ls)))

In [36]:
pars = []
for p in eng[0].findAll('p'):
    es = ' '.join(p.text.replace('\n',' ').split())
    es = re.sub(r'\. \. \.','',es)
    es = re.sub('[:;,]','',es)
    pars.append(len(sent_tokenize(es)))

In [37]:
l = list(zip(pars,lpars))

In [38]:
for i in range(len(l)):
    print(i,l[i])

0 (7, 8)
1 (6, 5)
2 (6, 7)
3 (3, 4)
4 (4, 3)
5 (8, 7)
6 (5, 7)
7 (4, 4)
8 (4, 4)
9 (8, 6)
10 (4, 5)
11 (8, 8)
12 (5, 6)
13 (6, 7)
14 (5, 6)
15 (3, 4)
16 (2, 4)
17 (5, 9)
18 (4, 4)
19 (5, 6)
20 (5, 5)
21 (5, 5)
22 (3, 3)
23 (3, 3)
24 (7, 7)
25 (10, 9)
26 (4, 4)
27 (4, 4)
28 (4, 4)
29 (3, 3)
30 (15, 16)
31 (5, 5)
32 (5, 5)
33 (3, 4)
34 (5, 2)
35 (4, 8)
36 (3, 3)
37 (5, 4)
38 (8, 7)
39 (18, 17)
40 (4, 4)
41 (6, 7)
42 (9, 10)
43 (12, 17)
44 (2, 3)
45 (4, 4)
46 (8, 8)
47 (7, 5)
48 (6, 6)
49 (6, 6)
50 (4, 3)
51 (8, 7)
52 (10, 8)
53 (3, 2)


In [218]:
' '.join(eng[2].findAll('p')[23].text.split())

'Having drawn out all his forces at the break of day, and marshaled them in a double line, he posted the auxiliaries in the center, and waited to see what measures the enemy would take. They, although on account of their great number and their ancient renown in war, and the small number of our men, they supposed they might safely fight, nevertheless considered it safer to gain the victory without any wound, by besetting the passes [and] cutting off the provisions: and if the Romans, on account of the want of corn, should begin to retreat, they intended to attack them while encumbered in their march and depressed in spirit [as being assailed while] under baggage. This measure being approved of by the leaders and the forces of the Romans drawn out, the enemy [still] kept themselves in their camp. Crassus having remarked this circumstance, since the enemy, intimidated by their own delay, and by the reputation [i.e. for cowardice arising thence] had rendered our soldiers more eager for fig

In [219]:
lat[2].findAll('p')[23].text

'Crassus equitum praefectos cohortatus, ut magnis praemiis pollicitationibusque suos excitarent, quid fieri vellet ostendit. Illi, ut erat imperatum, eductis iis cohortibus quae praesidio castris relictae intritae ab labore erant, et longiore itinere circumductis, ne ex hostium castris conspici possent, omnium oculis mentibusque ad pugnam intentis celeriter ad eas quas diximus munitiones pervenerunt atque his prorutis prius in hostium castris constiterunt quam plane ab his videri aut quid rei gereretur cognosci posset. Tum vero clamore ab ea parte audito nostri redintegratis viribus, quod plerumque in spe victoriae accidere consuevit, acrius impugnare coeperunt. Hostes undique circumventi desperatis omnibus rebus se per munitiones deicere et fuga salutem petere contenderunt. Quos equitatus apertissimis campis consectatus ex milium L numero, quae ex Aquitania Cantabrisque convenisse constabat, vix quarta parte relicta, multa nocte se in castra recepit.'

In [ ]:
es = ' '.join(ech_1[c][0].text.replace('\n',' ').split())
ls = lch_1[c][0].text.replace('\n', ' ')

ls = re.sub(r'\. \. \.','',ls)
es = re.sub(r'\. \. \.','',es)
ls = re.sub('[:;,]','',ls)
es = re.sub('[:;,]','',es)

epar = sent_tokenize(es)
lpar = s_tokenizer.tokenize_sentences(ls)

In [187]:
eng[2].findAll('p')[0]

<p>When <name type="pers">Caesar</name> was setting out for <name key="tgn,1000080" reg="Italy [12.833,42.833] (nation), Europe" type="place">Italy</name>, he
                    sent <name>Servius Galba</name> with the twelfth legion and part of the cavalry,
                    against the <name>Nantuates</name>, the <name>Veragri</name>, and
                        <name>Seduni</name>, who extend from the territories of the
                        <name>Allobroges</name>, and the lake of <name key="tgn,7007279" reg="  +Geneve [6.15,46.216] (inhabited place), Geneve, Switzerland, Europe " type="place">Geneva </name>, and the River <name key="tgn,7023890" reg="Rhone [4.833,43.333] (river), Europe" type="place">Rhone </name> to the top of the
                        <name key="tgn,7007746" reg="Alps (mountain system), Europe" type="place">Alps</name>. The reason for sending him was, that he desired that the pass
                    along the <name key="tgn,7007746" reg="Alps (mountain s

In [188]:
lat[2].findAll('p')[0]

<p><milestone n="1" unit="chapter"></milestone><milestone n="1" unit="section"></milestone>Cum in Italiam proficisceretur Caesar, Ser. Galbam cum legione XII. et parte equitatus in Nantuates, Veragros Sedunosque misit, qui a finibus Allobrogum et lacu Lemanno et flumine Rhodano ad summas Alpes pertinent. <milestone n="2" unit="section"></milestone>Causa mittendi fuit quod iter per Alpes, quo magno cum periculo magnisque cum portoriis mercatores ire consuerant, patefieri volebat. <milestone n="3" unit="section"></milestone>Huic permisit, si opus esse arbitraretur, uti in his locis legionem hiemandi causa conlocaret. <milestone n="4" unit="section"></milestone>Galba secundis aliquot proeliis factis castellisque compluribus eorum expugnatis, missis ad eum undique legatis obsidibusque datis et pace facta, constituit cohortes duas in Nantuatibus conlocare et ipse cum reliquis eius legionis cohortibus in vico Veragrorum, qui appellatur Octodurus hiemare; <milestone n="5" unit="section"></mil

In [39]:
lchaps = []
echaps = []
for i in range(len(lat)):
    try:
        p = get_pairs([lat[i]],[eng[i]],'div2','type','chapter')
        if type(p[0][0]) == list:
            lchaps.append([s[0] for s in p[0]])
            echaps.append([s[0] for s in p[1]])
        else:
            lchaps.append(p[0])
            echaps.append(p[1])
    except:
        print('error with {}'.format(i))

error with 0
error with 1
error with 2
error with 3
error with 4
error with 5
error with 6
error with 7


In [40]:
l = [0,3,4,5,7]
alignments = []
for b in l:
    print('b: ',b)
    lp = lat[b].findAll('p')
    ep = eng[b].findAll('p')
    for i in range(len(lp)):
    #for c in range(len(lat[b])):
        es = ' '.join(ep[i].text.replace('\n',' ').split())
        ls = lp[i].text.replace('\n', ' ')

        ls = re.sub(r'\. \. \.','',ls)
        es = re.sub(r'\. \. \.','',es)
        ls = re.sub('[:;,]','',ls)
        es = re.sub('[:;,]','',es)

        epar = sent_tokenize(es)
        lpar = s_tokenizer.tokenize_sentences(ls)
        alignments.append(get_sec_alignments(lpar,epar))
        #sec_alignments.append(get_sec_alignments(lpar[p],epar[p]))
        #alignments.append(sec_alignments)

b:  0
b:  3
b:  4
b:  5
b:  7


In [41]:
len(alignments)

250

In [42]:
count = 0
for a in alignments:
    for s in a:
        count += len(s)

In [43]:
count

2320

In [44]:
alignments[40]

[('Hac oratione habita mirum in modum conversae sunt omnium mentes summaque alacritas et cupiditas belli gerendi innata est princepsque X. legio per tribunos militum ei gratias egit quod de se optimum iudicium fecisset seque esse ad bellum gerendum paratissimam confirmavit.',
  'Upon the delivery of this speech the minds of all were changed in a surprising manner and the highest ardor and eagerness for prosecuting the war were engendered and the tenth legion was the first to return thanks to him through their military tribunes for his having expressed this most favorable opinion of them and assured him that they were quite ready to prosecute the war.'),
 ('Deinde reliquae legiones cum tribunis militum et primorum ordinum centurionibus egerunt uti Caesari satis facerent se neque umquam dubitasse neque timuisse neque de summa belli suum iudicium sed imperatoris esse existimavisse.',
  "Then the other legions endeavored through their military tribunes and the centurions of the principal c

In [46]:
f = open('aligned_sentences/caes_bg_sentences_03457.json','w') 
j = json.dumps(alignments)
f.write(j)
f.close()